In [71]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import jieba as jb
import re
import matplotlib.font_manager
from pylab import mpl
from matplotlib.font_manager import _rebuild
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression, RidgeClassifier


df = pd.read_excel('商品和服务税收分类编码表.xls',header=None)

df = df.drop(columns=[0,1,2,3,4,5,6,7,8,9,10],index=[0]) 
df = df.drop(index=[1]) 
df.columns=["code","secondCategory","firstCategory","info"]

d = {'category':df['firstCategory'].value_counts().index, 'count': df['firstCategory'].value_counts()}
df_cat = pd.DataFrame(data=d).reset_index(drop=True)

df = df.reset_index(drop=True)
df['multiLabel']=0
for index, i in enumerate(df["secondCategory"]):
    li = []
#     print(str(df["secondCategory"][index]))
    li.append(str(df["secondCategory"][index]))
    li.append(df["firstCategory"][index])
    df['multiLabel'][index]= li

df

/Users/zhangxuanting/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,code,secondCategory,firstCategory,info,multiLabel
0,1000000000000000000,货物,货物,NaN,"[货物, 货物]"
1,1010000000000000000,农、林、牧、渔业类产品,农产品,NaN,"[农、林、牧、渔业类产品, 农产品]"
2,1010100000000000000,农业产品,农业产品,NaN,"[农业产品, 农业产品]"
3,1010101000000000000,谷物,谷物,包括稻谷、小麦、玉米、谷子、高粱、大麦、燕麦、黑麦、荞麦、其他谷物，不包括种用谷物（详见10...,"[谷物, 谷物]"
4,1010101010000000000,稻谷,谷物,包括早籼稻、晚籼稻、中籼稻、粳稻、糯稻、其他稻谷。,"[稻谷, 谷物]"
...,...,...,...,...,...
4200,6080000000000000000,资产重组涉及的土地使用权,资产重组涉及的土地使用权,NaN,"[资产重组涉及的土地使用权, 资产重组涉及的土地使用权]"
4201,6090000000000000000,代理进口免税货物货款,代理进口免税货物货款,NaN,"[代理进口免税货物货款, 代理进口免税货物货款]"
4202,6100000000000000000,有奖发票奖金支付,有奖发票奖金,NaN,"[有奖发票奖金支付, 有奖发票奖金]"
4203,6110000000000000000,不征税自来水,不征税自来水,NaN,"[不征税自来水, 不征税自来水]"


In [72]:
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  

stopwords = stopwordslist("chineseStopWords.txt")

df['info'] = df['info'].apply(remove_punctuation)

df['info'] = df['info'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df


,code,secondCategory,firstCategory,info,multiLabel
0,1000000000000000000,货物,货物,nan,"[货物, 货物]"
1,1010000000000000000,农、林、牧、渔业类产品,农产品,nan,"[农、林、牧、渔业类产品, 农产品]"
2,1010100000000000000,农业产品,农业产品,nan,"[农业产品, 农业产品]"
3,1010101000000000000,谷物,谷物,稻谷 小麦 玉米 谷子 高粱 大麦 燕麦 黑麦 荞麦 谷物 种用 谷物 详见 101011901,"[谷物, 谷物]"
4,1010101010000000000,稻谷,谷物,早籼稻 晚 籼稻 中 籼稻 粳稻 糯稻 稻谷,"[稻谷, 谷物]"
...,...,...,...,...,...
4200,6080000000000000000,资产重组涉及的土地使用权,资产重组涉及的土地使用权,nan,"[资产重组涉及的土地使用权, 资产重组涉及的土地使用权]"
4201,6090000000000000000,代理进口免税货物货款,代理进口免税货物货款,nan,"[代理进口免税货物货款, 代理进口免税货物货款]"
4202,6100000000000000000,有奖发票奖金支付,有奖发票奖金,nan,"[有奖发票奖金支付, 有奖发票奖金]"
4203,6110000000000000000,不征税自来水,不征税自来水,nan,"[不征税自来水, 不征税自来水]"


In [90]:
X_train, X_test, y_train, y_test = train_test_split(df['info'], df['multiLabel'], random_state = 20)

cv = CountVectorizer(min_df=5,max_df=0.9,ngram_range=(1,2),token_pattern= '(\S+)')
feature = cv.fit_transform(X_train)
feature.shape


# tfidf = TfidfVectorizer(min_df=5,max_df=0.9,ngram_range=(1,2),token_pattern= '(\S+)')
# feature = tfidf.fit_transform(X_train)


(3153, 1320)

In [91]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

tags = df['multiLabel'].values
tag_dic={}
for tag_list in tags:
    for tag in tag_list:        
        if tag not in tag_dic:
            tag_dic[tag]=1
        else:
             tag_dic[tag]+=1
print(list(tag_dic.keys()))
#生成多标签的词袋矩阵list(tmp.keys())[0]
print(y_train)
mlb = MultiLabelBinarizer(classes=list(tag_dic.keys()))
y_t = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_test)
print(y_t.shape)
print(y_train[0])
print(y_t[0])
# y_train = pd.DataFrame(y_train)
# y_train

['货物', '农、林、牧、渔业类产品', '农产品', '农业产品', '谷物', '稻谷', '小麦', '玉米', '谷子', '高粱', '大麦', '燕麦', '黑麦', '荞麦', '其他谷物', '薯类', '马铃薯', '木薯', '甘薯', '其他薯类', '油料作物', '油料', '花生', '油菜籽', '芝麻', '其他油料', '干豆类', '干豆', '大豆', '绿豆', '其他杂豆', '棉花', '生麻', '糖料', '甘蔗', '甜菜', '其他糖料', '未加工烟草', '烟草', '未去梗烤烟叶', '其他未加工烟草', '饲料作物', '饲料牧草', '其他饲料作物', '水生植物类', '水生植物', '农作物副产品', '农副产品', '作物茎、秆、根', '其他农作物副产品', '蔬菜及食用菌', '萝卜等根菜类蔬菜', '蔬菜', '山药等薯芋类蔬菜', '葱蒜类蔬菜', '白菜类蔬菜', '芥菜类蔬菜', '甘蓝类蔬菜', '芹菜等叶菜类蔬菜', '瓜类蔬菜', '番茄等茄果类蔬菜', '菜豆等豆类蔬菜', '莲藕等水生蔬菜', '竹笋等多年生及杂类蔬菜', '香菇等食用菌', '食用菌', '绿豆芽等芽苗菜', '其他蔬菜', '花卉', '盆栽花', '鲜切花及花蕾', '切叶', '切枝', '干燥花', '盆景及园艺产品', '园艺产品', '盆栽观叶植物', '草皮', '草坪', '其他盆景及园艺产品', '水果及坚果', '水果（园林水果）', '水果', '苹果', '梨', '柑橘类水果', '葡萄', '香蕉', '菠萝', '龙眼', '荔枝', '枇杷', '红毛丹', '芒果', '橄榄', '无花果', '鳄梨', '番石榴', '山竹果', '杨桃', '莲雾', '火龙果', '西瓜', '哈密瓜', '华莱士瓜', '香瓜', '伊利沙白瓜', '木瓜', '樱桃', '枣', '红果', '柿子', '桃', '李子', '石榴', '杏', '杨梅', '草莓', '黑莓', '桑椹', '猕猴桃', '沙棘', '其他未列明水果', '干制水果', '食用坚果', '坚果', '椰子', '腰果', '核桃', '山核桃', '栗子', '松子', '榛子', '阿月浑子

2959                                 [其他金属处理机械, 金属处理机械]
1598                                     [草酸钐铕钆, 稀土化合物]
4051                                     [桥梁通行服务, 经营租赁]
1538                                      [醋酸镧铈, 稀土化合物]
2441                                       [铜丝制品, 金属制品]
4033                                 [有形动产融资租赁服务, 融资租赁]
3577                                  [绘图台及绘图机, 绘图测量仪器]
2187                                 [中小型型钢, 黑色金属冶炼压延品]
3673                                 [新型材料测试仪器, 光电测量仪器]
2758                            [玻璃、陶瓷制品生产专用设备, 玻璃陶瓷设备]
1515                                   [硝酸钬铒铥镱镥, 稀土化合物]
1162                                       [宠物用品, 宠物用品]
889                                    [油印蜡纸或胶印版纸, 纸制品]
2609                                       [液压辅件, 液压元件]
18                                           [其他薯类, 薯类]
1873                                       [中成药散剂, 中成药]
398                                            [饲料, 饲料]
1042                                    [电动游乐设备,

In [75]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
 
def print_evaluation_scores(y_val, predicted):
    accuracy=accuracy_score(y_val, predicted)
    f1_score_macro=f1_score(y_val, predicted, average='macro')
    f1_score_micro=f1_score(y_val, predicted, average='micro')
    f1_score_weighted=f1_score(y_val, predicted, average='weighted')
    print("accuracy:",accuracy)
    print("f1_score_macro:",f1_score_macro)
    print("f1_score_micro:",f1_score_micro)
    print("f1_score_weighted:",f1_score_weighted)
    
from sklearn.pipeline import Pipeline
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(min_df=5,max_df=0.9,ngram_range=(1,2),token_pattern= '(\S+)')),
                ('clf', OneVsRestClassifier(MultinomialNB())),
            ])
 
NB_pipeline.fit(X_train,y_train)
predicted = NB_pipeline.predict(X_test)
print(predicted)
print_evaluation_scores(y_val,predicted)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 13 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 23 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 30 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 35 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-p

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 368 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 371 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 372 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 374 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 380 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 747 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 748 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 749 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 759 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 765 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 932 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 935 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 936 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 938 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 946 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 1295 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 1301 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 1302 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 1308 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 1309 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4098 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4099 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4101 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4102 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4103 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4280 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4286 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4289 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4291 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4297 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


accuracy: 0.0
f1_score_macro: 0.0008366246734811052
f1_score_micro: 0.09907407407407409
f1_score_weighted: 0.06517311074940264


In [76]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(min_df=5,max_df=0.9,ngram_range=(1,2),token_pattern= '(\S+)')),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
 
SVC_pipeline.fit(X_train,y_train)
predicted = SVC_pipeline.predict(X_test)
# print(predicted)
print_evaluation_scores(y_val,predicted)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 13 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 23 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 30 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 35 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-p

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 227 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 229 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 230 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 234 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 235 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 409 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 410 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 411 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 412 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 415 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 677 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 679 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 682 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 683 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 684 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 829 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 834 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 837 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 846 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 847 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3843 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3846 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3847 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3852 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3853 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4034 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4035 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4036 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4044 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4045 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4222 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4227 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4229 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4230 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4231 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


accuracy: 0.005703422053231939
f1_score_macro: 0.011519950531051058
f1_score_micro: 0.24302954640033292
f1_score_weighted: 0.17639752149531412


In [77]:

LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(min_df=5,max_df=0.9,ngram_range=(1,2),token_pattern= '(\S+)')),
                ('clf', OneVsRestClassifier(LogisticRegression(), n_jobs=1)),
            ])
 
LogReg_pipeline.fit(X_train,y_train)
predicted = LogReg_pipeline.predict(X_test)
print_evaluation_scores(y_val,predicted)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 13 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 23 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 30 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 35 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-p

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 219 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 227 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 229 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 230 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 234 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 401 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 409 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 410 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 411 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 412 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 646 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 648 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 650 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 662 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 665 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 801 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 802 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 803 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 808 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 815 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/s

...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3795 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3799 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3803 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3804 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3815 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4015 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4016 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4027 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4034 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4035 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4199 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4201 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4202 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4203 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 4206 is present in all training examples.
  str(classes[c]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


accuracy: 0.0
f1_score_macro: 0.0004910148415315442
f1_score_micro: 0.06701274185936762
f1_score_weighted: 0.04854942227089637
